In [2]:
## idg forked from @levanpon1009 on kaggle.com
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from itertools import chain
from glob import glob
import pandas as pd
import numpy as np
import os

2021-10-13 06:54:50.481095: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


### const

In [3]:
CWD = os.getcwd()
PROJECT_NAME = 'Pneumonia-Classification'
BATCH_SIZE = 16
DIM = (224, 224)

In [4]:
print(CWD)

/home/jinen/git/Pneumonia-Classification/task-2-data-processing/src/notebook


### load dataset

In [5]:
DATA_DIR = CWD[:40] + '/data/'
df = pd.read_csv(DATA_DIR + 'Data_Entry_2017.csv') # TEST '/home/jinen/git/Pneumonia-Classification/data/
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143


In [6]:
data_image_paths = {os.path.basename(x): x for x in glob(os.path.join(DATA_DIR, 'images*', '*', '*.png'))}

In [7]:
df['path'] = df['Image Index'].map(data_image_paths.get)

In [8]:
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],path
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,/home/jinen/git/Pneumonia-Classification/data/...
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,/home/jinen/git/Pneumonia-Classification/data/...
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,/home/jinen/git/Pneumonia-Classification/data/...
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,/home/jinen/git/Pneumonia-Classification/data/...
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,/home/jinen/git/Pneumonia-Classification/data/...


In [9]:
labels = []
for i in df['Finding Labels']:
    if 'Pneumonia' in i:
        labels.append(1)
    else:
        labels.append(0)
df['Finding Labels'] = np.array(labels) 
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],path
0,00000001_000.png,0,0,1,58,M,PA,2682,2749,0.143,0.143,/home/jinen/git/Pneumonia-Classification/data/...
1,00000001_001.png,0,1,1,58,M,PA,2894,2729,0.143,0.143,/home/jinen/git/Pneumonia-Classification/data/...
2,00000001_002.png,0,2,1,58,M,PA,2500,2048,0.168,0.168,/home/jinen/git/Pneumonia-Classification/data/...
3,00000002_000.png,0,0,2,81,M,PA,2500,2048,0.171,0.171,/home/jinen/git/Pneumonia-Classification/data/...
4,00000003_000.png,0,0,3,81,F,PA,2582,2991,0.143,0.143,/home/jinen/git/Pneumonia-Classification/data/...


In [10]:
df['path'] = df['path'].astype('str')

In [11]:
train_df, valid_df = train_test_split(df, test_size=0.20, random_state=2018, stratify=df['Finding Labels'])

In [14]:
core_generator = ImageDataGenerator(rescale=1 / 255,
                                    samplewise_center=True,
                                    samplewise_std_normalization=True,
                                    horizontal_flip=True,
                                    vertical_flip=False)

training_generator = core_generator.flow_from_dataframe(dataframe=train_df,
                                                        directory=None,
                                                        x_col='path',
                                                        y_col='Finding Labels',
                                                        class_mode='raw',
                                                        batch_size=BATCH_SIZE,
                                                        target_size=DIM)

validation_generator = core_generator.flow_from_dataframe(dataframe=valid_df,
                                                          directory=None,
                                                          x_col='path',
                                                          y_col='Finding Labels',
                                                          class_mode='raw',
                                                          batch_size=BATCH_SIZE,
                                                          target_size=DIM)

test_generator = core_generator.flow_from_dataframe(dataframe=valid_df,
                                                    directory=None,
                                                    x_col='path',
                                                    y_col='Finding Labels',
                                                    class_mode='raw',
                                                    batch_size=BATCH_SIZE,
                                                    target_size=DIM)

Found 89696 validated image filenames.
Found 22424 validated image filenames.
Found 22424 validated image filenames.
